In [37]:
# external
import datasets as ds
from transformers import pipeline
import importlib
import os
import torch
import transformers
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from functools import partial
import numpy as np

# internal
from redditqa.data.smart_filter import question_grading_map, question_filter, answer_grading_map, answer_filter
from redditqa.data import qa_generation
from redditqa.data.util import mask_links

importlib.reload(qa_generation)

<module 'redditqa.data.qa_generation' from '/workspaces/reddit_qa/redditqa/data/qa_generation.py'>

In [2]:
torch.cuda.is_available()

True

In [3]:
model_id = "HuggingFaceH4/zephyr-7b-beta"
cache_dir = "/scratch1/ssawicki/cache"

### Load model

In [4]:
# quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        low_cpu_mem_usage=True,
        quantization_config=bnb_config,
        cache_dir=cache_dir
    )

Loading checkpoint shards: 100%|██████████| 8/8 [01:03<00:00,  7.90s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

### Load dataset

In [8]:
dataset = ds.load_from_disk("/scratch1/redditqa/cached_datasets/AskHistorians.jsonl")
# Preprocessing
dataset = dataset.map(mask_links)

In [9]:
dataset

Dataset({
    features: ['question_created_utc', 'question_retrieved_on', 'question_deleted', 'question_title', 'question_selftext', 'question_score', 'question_char_length', 'question_selftext_char_length', 'answers'],
    num_rows: 116452
})

In [10]:
dataset = dataset.filter(lambda x: len(x['question_title']) >= 55)

In [11]:
dataset = dataset.filter(lambda x: x['question_score'] >= 4)

In [10]:
dataset = ds.load_from_disk("/scratch1/redditqa/cached_datasets/AskHistorians_question_filtered.jsonl")
question_filter = partial(question_filter, accepted_token_str=['y', 'yes'])
dataset = dataset.filter(question_filter)
dataset[1000:1100]['question_title']

['In pre-modern warfare, how did opposing armies handle battle field casualties in multi-day engagements?',
 'Anybody familiar with how black troops were treated by the American military in WWII?',
 'Did King Richard III murder the princes in the Tower of London?',
 'How were weapon control laws implemented throughout history? Did strict laws ever really exist banning civilian use of things like swords, bows, and gunpowder?',
 'A few questions about the end of the civil war and slavery, inspired by the movie *Lincoln*',
 'The Polar Bear Expedition of 1918 was basically an allied invasion of Russia, why did it have such limited ramifications?',
 'Is there any truth to the Soviet Union and China having a "location ambiguity" pact about nuclear weapons during the Cold War?',
 'Why did the Russians take Prussia from the Germans after WW2?',
 'How does a society typically recover from a state of poor law enforcement?',
 'During WW2, how would non-combatant aircraft and ships travel safely t

In [5]:
dataset.save_to_disk("/scratch1/redditqa/cached_datasets/AskHistorians_question_filtered.jsonl")

Saving the dataset (1/1 shards): 100%|██████████| 1/1 [00:00<00:00, 177.16 examples/s]


In [15]:
dataset.info.features

{'question_created_utc': Value(dtype='int64', id=None),
 'question_retrieved_on': Value(dtype='int64', id=None),
 'question_deleted': Value(dtype='bool', id=None),
 'question_title': Value(dtype='string', id=None),
 'question_selftext': Value(dtype='string', id=None),
 'question_score': Value(dtype='int64', id=None),
 'question_char_length': Value(dtype='int64', id=None),
 'question_selftext_char_length': Value(dtype='int64', id=None),
 'answers': [{'answer_body': Value(dtype='string', id=None),
   'answer_char_length': Value(dtype='int64', id=None),
   'answer_created_utc': Value(dtype='int64', id=None),
   'answer_deleted': Value(dtype='bool', id=None),
   'answer_id': Value(dtype='string', id=None),
   'answer_retrieved_on': Value(dtype='int64', id=None),
   'answer_score': Value(dtype='int64', id=None)}]}

### LLM Filtering

Question filtering:
- Few shot classification: define what a well written question is, show some good and bad examples.

Answer filter:
- Few shot classification: define what a well written answer is, show good & bad examples.

#### Question filter

In [44]:
question_grading = partial(question_grading_map, pipeline=pipe, verbose=True)
question_filter = partial(question_filter, accepted_token_str=['y', 'yes'])

In [45]:
example_ds = dataset.select(list(range(50,80)))
example_ds

Dataset({
    features: ['question_created_utc', 'question_retrieved_on', 'question_deleted', 'question_title', 'question_selftext', 'question_score', 'question_char_length', 'question_selftext_char_length', 'answers'],
    num_rows: 30
})

In [48]:
# grading the questions
example_ds = example_ds.map(question_grading)

Map:   3%|▎         | 1/30 [00:00<00:06,  4.51 examples/s]

{'graded_output': [{'token_id': tensor(9780), 'token_str': 'yes', 'probability': 0.0010224399}, {'token_id': tensor(1510), 'token_str': 'no', 'probability': 4.9931226e-10}, {'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.85968566}, {'token_id': tensor(2501), 'token_str': 'No', 'probability': 1.7314133e-07}, {'token_id': tensor(28724), 'token_str': 'y', 'probability': 4.893228e-08}, {'token_id': tensor(28711), 'token_str': 'n', 'probability': 1.4886707e-09}, {'token_id': tensor(5613), 'token_str': 'generated: Yes', 'probability': 0.85968566}]}


Map:   7%|▋         | 2/30 [00:00<00:06,  4.59 examples/s]

{'graded_output': [{'token_id': tensor(9780), 'token_str': 'yes', 'probability': 0.012283739}, {'token_id': tensor(1510), 'token_str': 'no', 'probability': 1.866186e-09}, {'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.7902418}, {'token_id': tensor(2501), 'token_str': 'No', 'probability': 8.2891866e-08}, {'token_id': tensor(28724), 'token_str': 'y', 'probability': 4.1281754e-07}, {'token_id': tensor(28711), 'token_str': 'n', 'probability': 3.214851e-09}, {'token_id': tensor(5613), 'token_str': 'generated: Yes', 'probability': 0.7902418}]}


Map:  10%|█         | 3/30 [00:00<00:05,  4.63 examples/s]

...

{'graded_output': [{'token_id': tensor(9780), 'token_str': 'yes', 'probability': 0.0012166302}, {'token_id': tensor(1510), 'token_str': 'no', 'probability': 2.367969e-10}, {'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.86142224}, {'token_id': tensor(2501), 'token_str': 'No', 'probability': 1.4723997e-07}, {'token_id': tensor(28724), 'token_str': 'y', 'probability': 2.2980423e-08}, {'token_id': tensor(28711), 'token_str': 'n', 'probability': 1.1642753e-09}, {'token_id': tensor(5613), 'token_str': 'generated: Yes', 'probability': 0.86142224}]}


Map:  97%|█████████▋| 29/30 [00:06<00:00,  4.64 examples/s]

{'graded_output': [{'token_id': tensor(9780), 'token_str': 'yes', 'probability': 1.27161675e-05}, {'token_id': tensor(1510), 'token_str': 'no', 'probability': 1.1087928e-07}, {'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.009927167}, {'token_id': tensor(2501), 'token_str': 'No', 'probability': 5.0886258e-05}, {'token_id': tensor(28724), 'token_str': 'y', 'probability': 1.5514799e-07}, {'token_id': tensor(28711), 'token_str': 'n', 'probability': 4.4624613e-08}, {'token_id': tensor(2198), 'token_str': 'generated: As', 'probability': 0.5091666}]}


Map: 100%|██████████| 30/30 [00:06<00:00,  4.62 examples/s]

{'graded_output': [{'token_id': tensor(9780), 'token_str': 'yes', 'probability': 0.0006323905}, {'token_id': tensor(1510), 'token_str': 'no', 'probability': 9.781082e-09}, {'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.5660184}, {'token_id': tensor(2501), 'token_str': 'No', 'probability': 1.6621306e-06}, {'token_id': tensor(28724), 'token_str': 'y', 'probability': 1.6297389e-07}, {'token_id': tensor(28711), 'token_str': 'n', 'probability': 1.3815863e-08}, {'token_id': tensor(5613), 'token_str': 'generated: Yes', 'probability': 0.5660184}]}


In [13]:
# filtering quetions
example_ds = example_ds.filter(question_filter)

Filter: 100%|██████████| 30/30 [00:00<00:00, 5564.46 examples/s]


In [14]:
# remaining questions
example_ds["question_title"]

['Was Dwight Eisenhower consulted by JFK during the Cuban Missile Crisis? ',
 'Religious sects of Reformation England: Who were they and what did they believe?',
 'Reddit, what were the perspectives of the British and the Boers during the Second Boer War ?',
 'Were the fire bombings on Dresden from Allied forces during WWII necessary?',
 'What was the role of dogs in the Roman military?',
 'What did pre-modern racism look like?',
 'How did the Roman government react to the eruption of Mt. Vesuvius in AD 79?',
 'Do you guys know of any instance when a revolution has succeeded without foreign intervention?',
 'Is there any other instance in history where a different race of people were imported into a land as slaves and eventually became fully assimilated as equals into society? If so how long did it take? ',
 'What was the longest period of time where the US was not at war, or actively involved in combat?']

#### Answer filter

In [15]:
answer_grading = partial(answer_grading_map, pipeline=pipe, verbose=True)
answer_filter = partial(answer_filter, accepted_token_str=['y', 'yes'])

In [16]:
example_ds = example_ds.map(answer_grading)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.7641037}]}


Map:  10%|█         | 1/10 [00:00<00:05,  1.72 examples/s]

{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.7315472}]}
{'graded_output': [{'token_id': tensor(2301), 'token_str': 'Is', 'probability': 0.46591264}]}


Map:  20%|██        | 2/10 [00:01<00:05,  1.58 examples/s]

...

{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.94298965}]}
{'graded_output': [{'token_id': tensor(2501), 'token_str': 'No', 'probability': 0.6223856}]}
{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.38676727}]}
{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.91877407}]}
{'graded_output': [{'token_id': tensor(2301), 'token_str': 'Is', 'probability': 0.44995272}]}
{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.5059207}]}
{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.81613564}]}
{'graded_output': [{'token_id': tensor(1014), 'token_str': 'The', 'probability': 0.35310444}]}
{'graded_output': [{'token_id': tensor(2501), 'token_str': 'No', 'probability': 0.49348438}]}
{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.91818935}]}
{'graded_output': [{'token_id': tensor(2301), 'token_st

Map:  90%|█████████ | 9/10 [00:18<00:03,  3.66s/ examples]

{'graded_output': [{'token_id': tensor(2501), 'token_str': 'No', 'probability': 0.9745897}]}
{'graded_output': [{'token_id': tensor(2301), 'token_str': 'Is', 'probability': 0.43321145}]}


Map: 100%|██████████| 10/10 [00:18<00:00,  1.86s/ examples]

{'graded_output': [{'token_id': tensor(5613), 'token_str': 'Yes', 'probability': 0.89880574}]}


In [17]:
# filter the answers
example_ds = example_ds.map(answer_filter)

Map: 100%|██████████| 10/10 [00:00<00:00, 1067.09 examples/s]


In [18]:
# remaining answers
example_ds["answers"]

[[{'answer_body': "Generally speaking, yes. Presidents often meet with with ex-presidents, regardless of political alignment.\n\nSpecifically for this case, yes. A lot. Eisenhower was President during the build-up to the Cuban Missile Crisis and widely renowned as an excellent general. Here is a recording of one such phone call:\n\n[LINK]\n\nHere's another:\n\n[LINK]",
   'answer_char_length': 407,
   'answer_created_utc': 1347938144000,
   'answer_deleted': False,
   'answer_id': 'c69qhes',
   'answer_retrieved_on': 1429926087000,
   'answer_score': 2,
   'graded_output': [{'probability': 0.7641037106513977,
     'token_id': 5613,
     'token_str': 'Yes'}]},
  {'answer_body': "Interestingly enough, Eisenhower was sharply criticized shortly before the Crisis because he made a statement criticizing Kennedy's foreign policy, thus breaking the informal rule against past presidents criticizing sitting ones. As Barrel-rider stated, however, Eisenhower offered his support during the Crisis."

### Toxicity

Currently not in use.

In [5]:
toxicity_pipe = pipeline("text-classification", model="tomh/toxigen_roberta", device=0)

In [13]:
toxicity_pipe('I love ML', top_k=None)

[{'label': 'LABEL_0', 'score': 0.9992640614509583},
 {'label': 'LABEL_1', 'score': 0.0007359233568422496}]

In [6]:
def run_toxicity_pipe(text):
    try: 
        result = toxicity_pipe(text, top_k=None)
        result = [r for r in result if r['label'] == 'LABEL_1'][0]
    except:
        return 0.5

    return result['score']

In [11]:
for answer in dataset[65]['answers']:
    print(run_toxicity_pipe(answer["answer_body"]))

0.0006564322975464165
0.0007360000745393336
0.0007385569042526186
0.000814662838820368
